In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np


from tqdm.notebook import tqdm

In [2]:
def calculate_euclidean_distance(row):
    return np.sqrt(
        (row["gaze_angle_x"] - row["target_angle_x"]) ** 2
        + (row["gaze_angle_y"] - row["target_angle_y"]) ** 2
    )


def calculate_accuracy_and_std(df):
    eye_tracker = df["eye_tracker"].iloc[0]
    
    accuracy_results = []
    std_results = []

    for trial in df["trial_number"].unique():
        trial_data = df[df["trial_number"] == trial]


        errors = trial_data.apply(calculate_euclidean_distance, axis=1)
        accuracy = errors.mean()
        std = errors.std()

        n_samples = len(trial_data)

        # Append to accuracy DataFrame
        accuracy_results.append({
            "eye_tracker": trial_data["eye_tracker"].iloc[0],
            "participant_id": trial_data["participant_id"].iloc[0],
            "trial_number": trial,
            "trial_condition": trial_data["trial_condition"].iloc[0],
            "accuracy": accuracy,
            "n_samples": n_samples,
        })

        # Append to std DataFrame
        std_results.append({
            "eye_tracker": df["eye_tracker"].iloc[0],
            "participant_id": df["participant_id"].iloc[0],
            "trial_number": trial,
            "trial_condition":trial_data["trial_condition"].iloc[0],
            "std": std,
            "n_samples": n_samples,
        })

    accuracy_df = pd.DataFrame(accuracy_results)
    std_df = pd.DataFrame(std_results)

    return accuracy_df, std_df


def calculate_apparent_gaze_shift(df):
    eye_tracker = df["eye_tracker"].iloc[0]

    results = []

    # Get all trials
    trials = sorted(df["trial_number"].unique())

    # Process pairs of trials (odd-even pairs)
    for i in range(0, len(trials) - 1, 2):
        dilated_trial = trials[i]  
        constricted_trial = trials[i + 1] 

        # Get data for each trial
        constricted_data = df[df["trial_number"] == constricted_trial]
        dilated_data = df[df["trial_number"] == dilated_trial]
        
        # Calculate mean gaze positions
        constricted_x = constricted_data["gaze_angle_x"].mean()
        constricted_y = constricted_data["gaze_angle_y"].mean()
        dilated_x = dilated_data["gaze_angle_x"].mean()
        dilated_y = dilated_data["gaze_angle_y"].mean()

        # Calculate distance
        x_diff = dilated_x - constricted_x
        y_diff = dilated_y - constricted_y
        distance = np.sqrt(x_diff**2 + y_diff**2)
        results.append(
            {
                "eye_tracker": df["eye_tracker"].iloc[0],
                "participant_id": df["participant_id"].iloc[0],
                "dilated_trial": dilated_trial,
                "constricted_trial": constricted_trial,
                "apparent_gaze_shift": distance,
                "constricted_samples": len(constricted_data),
                "dilated_samples": len(dilated_data),
            }
        )

    # Convert to DataFrame
    results_df = pd.DataFrame(results)

    return results_df


def calculate_rms_s2s(df):
    eye_tracker = df["eye_tracker"].iloc[0]

    results = []

    for trial in df["trial_number"].unique():
        trial_data = df[df["trial_number"] == trial]

        # Compute consecutive gaze shifts
        dx = np.diff(trial_data['gaze_angle_x'])
        dy = np.diff(trial_data['gaze_angle_y'])
        distances = np.sqrt(dx**2 + dy**2)

        rms_s2s_degrees = np.sqrt(np.mean(distances ** 2))

        results.append({
            "eye_tracker": trial_data["eye_tracker"].iloc[0],
            "participant_id": trial_data["participant_id"].iloc[0],
            "trial_number": trial,
            "trial_condition": trial_data["trial_condition"].iloc[0],
            "rms_s2s": rms_s2s_degrees,
            "n_samples": len(trial_data)
        })

    return pd.DataFrame(results)


In [27]:
project_dir_path = Path(r"/Users/salari/Dropbox/DC3/Dev/psa_data_quality")
dataset_dir_path = project_dir_path / "data"


# Get all eye trackers data directories
eye_trackers = ["EyeLink 1000 Plus", "Pupil Core", "SMI ETG", "Pupil Neon", "Tobii Glasses 2"]
data_paths = []
for participant_dir in dataset_dir_path.iterdir():
    if participant_dir.is_dir():
        for eye_tracker in eye_trackers:
            data_path = participant_dir / eye_tracker / "data.csv"
            if data_path.exists():
                data_paths.append(data_path)

In [28]:
accuracy_list = []
std_list = []
rms_list = []
apparent_gaze_shift_list = []

for data_path in tqdm(data_paths, desc="Processing", unit="recording"):

    df = pd.read_csv(data_path)
    acc_df, std_df = calculate_accuracy_and_std(df)
    accuracy_list.append(acc_df)
    std_list.append(std_df)

    rms_list.append(calculate_rms_s2s(df))
    apparent_gaze_shift_list.append(calculate_apparent_gaze_shift(df))



# Combine all results into final DataFrames
accuracy_df = pd.concat(accuracy_list, ignore_index=True)
std_df = pd.concat(std_list, ignore_index=True)
rms_df = pd.concat(rms_list, ignore_index=True)
apparent_gaze_shift_df = pd.concat(apparent_gaze_shift_list, ignore_index=True)

# Sort
accuracy_df = accuracy_df.sort_values(by=["eye_tracker", "participant_id", "trial_number"])
std_df = std_df.sort_values(by=["eye_tracker", "participant_id", "trial_number"])
rms_df = rms_df.sort_values(by=["eye_tracker", "participant_id", "trial_number"])
apparent_gaze_shift_df = apparent_gaze_shift_df.sort_values(by=["eye_tracker", "participant_id"])  


Processing:   0%|          | 0/102 [00:00<?, ?recording/s]

In [29]:
# Define output directory
output_dir = project_dir_path / "quality_metrics"
output_dir.mkdir(exist_ok=True)  # Create the directory if it doesn't exist

# Save each DataFrame to a CSV file
accuracy_df.to_csv(output_dir / "accuracy.csv", index=False)
std_df.to_csv(output_dir / "std.csv", index=False)
rms_df.to_csv(output_dir / "rms_s2s.csv", index=False)
apparent_gaze_shift_df.to_csv(output_dir / "apparent_gaze_shift.csv", index=False)

print(f"Saved CSV files to: {output_dir}")

Saved CSV files to: /Users/salari/Dropbox/DC3/Dev/psa_data_quality/quality_metrics


In [22]:
# accuracy_df[(accuracy_df['eye_tracker'] == 'Pupil Core') & (accuracy_df['participant_id'] == 141)]
